<a href="https://colab.research.google.com/github/misabhishek/gcp-iam-recommender/blob/main/IAM_Recommender_demo_for_GCS_buckets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IAM Recommender demo for GCS buckets

The purpose of this colab is to give you a glimse of what can you do with GCS buckets' recommendations in achieving least privillege. 

You will learn how to get IAM Recommendations for GCS buckets using gcloud, BigQuery export, and Curl API. 

## Before you begin
1. Be familiar with [API authentication flow](https://cloud.google.com/docs/authentication).
2. Understand the [IAM recommender](https://cloud.google.com/iam/docs/recommender-overview) and [enable its API](https://cloud.google.com/recommender/docs/enabling?hl=en). 
3. Make sure you have the [IAM permissions](https://cloud.google.com/iam/docs/recommender-managing#required-permissions) that you need for work with the IAM recommender. **You need these permissions on a project, folder, or organization that contains the storage bucket whose recommendations you want to manage.**
4. Review the IAM recommender best practices.

**Note:** Please take this [tutorial](https://colab.research.google.com/?utm_source=scs-index) if you are not familiar with colab.

## Authorize the colab

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()
print("authorized")

authorized


In [3]:
#@title Enter the configuration that will be used throughout the colab
project = "your-project-id (the project where the buckets resides)" #@param {type:"string"}
location = "us (location of the buckets)" #@param {type:"string"}
billing_project = "your-billing-project-id (the project that you used for onboarding the private preview )" #@param {type:"string"}

**Note:** To execute shell command in colab, just prepend it with exclamation mark (!). For example, one can execute `ls` command as below in colab. 

```
!ls 
```

We used this trick extensively to run `gcloud` and `curl` command.

# gcloud demo

In this demo, we present how to access the IAM recommendations and IAM Policy insights for GCS buckets using gcloud.

## Recommendation

### All bucket recommendations for a given project and location

In [ ]:
!gcloud recommender recommendations list \
--location="{location}" \
--recommender=google.iam.policy.Recommender \
--project="{project}" \
--format="table[box,title='IAM Recommendations at GCS bucket level'](targetResources.basename().flatten():sort=1:label=BUCKET,content.overview.member,content.overview.removedRole,content.overview.addedRoles,priority)" \
--billing-project="{billing_project}" \

### Recommendations to curtail public access on GCS buckets

In [ ]:
!gcloud recommender recommendations list \
--location="{location}" \
--recommender=google.iam.policy.Recommender \
--project="{project}" \
--billing-project="{billing_project}" \
--format="table[box,title='IAM Recommendations for curtailing public access of GCS buckets'](targetResources.basename().flatten():sort=1:label=BUCKET,content.overview.member,content.overview.removedRole,content.overview.addedRoles,priority)" \
--filter="content.overview.member:allUsers OR content.overview.member:allAuthenticatedUsers"

### Recommendations to curtail cross-project access 

In [ ]:
!gcloud recommender recommendations list \
--location="{location}" \
--recommender=google.iam.policy.Recommender \
--project="{project}" --format=json \
--billing-project="{billing_project}" \
--format="table[box,title='IAM Recommendations for curtailing public access of GCS buckets'](targetResources.basename().flatten():sort=1:label=BUCKET,content.overview.member,content.overview.removedRole,content.overview.addedRoles,priority)" \
--filter="content.overview.member:project* AND NOT content.overview.member:{project}"

### Recommendations to curtail default access



In [ ]:
!gcloud recommender recommendations list \
--location="{location}" \
--recommender=google.iam.policy.Recommender \
--project="{project}" --format=json \
--billing-project="{billing_project}" \
--format="table[box,title='IAM Recommendations for curtailing public access of GCS buckets'](targetResources.basename().flatten():sort=1:label=BUCKET,content.overview.member,content.overview.removedRole,content.overview.addedRoles,priority)" \
--filter="content.overview.member:project*{project}"

## Insight

### All bucket insights for a given project and location

In [ ]:
!gcloud recommender insights list \
--insight-type=google.iam.policy.Insight \
--project="{project}" \
--billing-project="{billing_project}" \
--location="{location}" \
--format="table[box,title='IAM Policy Insight at GCS bucket level'](targetResources.basename().flatten():sort=1:label=BUCKET,content.member,content.role,content.currentTotalPermissionsCount,content.exercisedPermissions.len(),content.exercisedPermissions.permission,severity)" \

### Insights about public access

In [ ]:
!gcloud recommender insights list \
--insight-type=google.iam.policy.Insight \
--project="{project}" \
--billing-project="{billing_project}" \
--location="{location}" \
--format="table[box,title='IAM Policy Insight at GCS bucket level'](targetResources.basename().flatten():sort=1:label=BUCKET,content.member,content.role,content.currentTotalPermissionsCount,content.exercisedPermissions.len(),content.exercisedPermissions.permission,severity)" \
--filter="content.member:allUsers OR content.member:allAuthenticatedUsers"


### Insight about cross-project access

In [ ]:
!gcloud recommender insights list \
--insight-type=google.iam.policy.Insight \
--project="{project}" \
--billing-project="{billing_project}" \
--location="{location}" \
--format="table[box,title='IAM Policy Insight at GCS bucket level'](targetResources.basename().flatten():sort=1:label=BUCKET,content.member,content.role,content.currentTotalPermissionsCount,content.exercisedPermissions.len(),content.exercisedPermissions.permission,severity)" \
--filter="content.member:project* AND NOT content.member:{project}"

### Insight about default access

In [ ]:
!gcloud recommender insights list \
--insight-type=google.iam.policy.Insight \
--project="{project}" \
--billing-project="{billing_project}" \
--location="{location}" \
--format="table[box,title='IAM Policy Insight at GCS bucket level'](targetResources.basename().flatten():sort=1:label=BUCKET,content.member,content.role,content.currentTotalPermissionsCount,content.exercisedPermissions.len(),content.exercisedPermissions.permission,severity)" \
--filter="content.member:project*{project}"

# BigQuery Export 

## For an organization level view of IAM recommendations and policy insights

1. Please enter the project that is storing BigQuery dataset for IAM Recommendations
2. Please enter the date (Ex - 2021-12-03)

In [ ]:
#@title Configuration for BigQuery export of GCS bucket recommendations
bigquery_export_project = "Enter the project used for BigQuery export" #@param{type:"string"}
date="Enter the snapshot date of the recommendations" #@param{type:"string"}
bigquery_dataset = "Enter the respective BigQuery Dataset" #@param{type:"string"}

## Organization Level View 

In [ ]:
from google.cloud import bigquery

bigquery_client = bigquery.Client(project=bigquery_export_project)

In [ ]:
query_to_see_all_recommendations_from_biqquery_export = f"""
SELECT 
  ancestors.organization_id,
  ancestors.folder_ids,
  cloud_entity_id AS project_number,
  location,
  SPLIT(target_resources[OFFSET(0)], "/")[OFFSET(3)] AS bucket_name,
  JSON_QUERY(recommendation_details, "$.overview.member") AS user,
  JSON_QUERY(recommendation_details, "$.overview.removedRole") AS removed_role,
  JSON_QUERY(recommendation_details, "$.overview.addedRoles") AS added_roles,
  JSON_QUERY(primary_impact.security_projection.details_json, "$.revokedIamPermissionsCount") 
      AS revoked_permission_count,
FROM 
  `{bigquery_dataset}`
WHERE
   recommender = 'google.iam.policy.Recommender'
    AND
  location != "global"
    AND
  DATE(_PARTITIONTIME) = "{date}"
"""

In [ ]:
(
    bigquery_client.query(
        query_to_see_all_recommendations_from_biqquery_export)
    .to_dataframe()
)

## Order projects based on impact of GCS bucket recommendations

In [ ]:
query_to_see_overgranted_projects_for_GCS_bucket_recommendations = f"""
SELECT 
  cloud_entity_id AS project_number,
  SUM(
    CAST(
      JSON_VALUE(primary_impact.security_projection.details_json, 
                 "$.revokedIamPermissionsCount") 
      AS FLOAT64)
    ) AS revoked_permission_count,
FROM 
  `{bigquery_dataset}`
WHERE
   recommender = 'google.iam.policy.Recommender'
    AND
  location != "global"
    AND
  DATE(_PARTITIONTIME) = "{date}"
GROUP BY cloud_entity_id
ORDER BY revoked_permission_count DESC
"""

In [ ]:
(
    bigquery_client.query(
        query_to_see_overgranted_projects_for_GCS_bucket_recommendations)
    .to_dataframe()
)

## Track progress of least privillege

In [ ]:
query_to_measure_progress_of_reduced_overgranting = f"""
SELECT
  DATE(_PARTITIONTIME) AS date,
  cloud_entity_id AS project_number,
  SUM(
    CAST(
      JSON_VALUE(primary_impact.security_projection.details_json, 
                 "$.revokedIamPermissionsCount") 
      AS FLOAT64)
    ) AS revoked_permission_count,
FROM 
  `{bigquery_dataset}`
WHERE
   recommender = 'google.iam.policy.Recommender'
    AND
  location != "global"
GROUP BY cloud_entity_id, DATE(_PARTITIONTIME)
"""

In [ ]:
(
    bigquery_client.query(
        query_to_measure_progress_of_reduced_overgranting)
    .to_dataframe()
    .set_index("date")
    .groupby("project_number")["revoked_permission_count"]
    .plot(legend=True, figsize=(10,5), rot=45, ylabel="Excess permissions count",
          title="Least privillege of IAM policies at GCS bucket level")
);

# API using curl


Use curl to get recommendations for a particular bucket

In [ ]:
bucket_name = "Enter the name your bucket for which you want recommendations." #@param{type:"string"}

In [ ]:
!curl -X GET \
-H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
-H "x-goog-user-project: {billing_project}" \
"https://recommender.googleapis.com/v1alpha2/projects/{project}/locations/{location}/recommenders/google.iam.policy.Recommender/recommendations?"\
"pageSize=10&"\
"filter=targetResources://storage.googleapis.com/ds-demo-vpc-sc-bucket"\